# SoftMax Regression

In [ ]:
#Imports

import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt

## Training data

In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [ ]:
# Plotting some examples
fig = plt.figure(figsize=(10,7))
for i in range(15):
    ax = fig.add_subplot(3, 5, i+1)
    ax.imshow(train_X[i], cmap=plt.get_cmap('gray'))
    ax.set_title('Label (y): {y}'.format(y=train_y[i]))
    plt.axis('off')

In [ ]:
X_train = train_X.reshape(train_X.shape[0], 28*28)
X_test = test_X.reshape(test_X.shape[0], 28*28)

In [ ]:
#Normalising
X_train = X_train/255
X_test = X_test/255

* For softax regression, label vector y needs to be represented as one-hot encoding

In [ ]:
def convert_to_one_hot_encoding(y, k):
    #Intitalising with zeros
    y_one_hot = np.zeros((len(y), k))

    #Putting 1 for column where the label is
    y_one_hot[np.arange(len(y)), y] = 1
    return y_one_hot


## Model: Softmax regression Implementation

### STEP 1
In vectorised form, we implement: $Z = XW + b$

In [ ]:
def linear_combination(X, w, b):
    return X @ w + b

### Step 2
Calculate softmax for each class 
$$
Softmax(Z) = \frac{exp(Z)}{\sum exp(Z)}
$$

In [ ]:
def softmax(z):
    # z--> linear part
    # Subtracting the max of z fo numerical stability.
    exp = np.exp(z - np.max(z))

    #Calculating softmax for all example
    for i in range(len(z)):
        exp[i] /= np.sum(exp[i])

    return exp

## Loss function and Optimisation
*  Categorical cross entropy loss

$$
J(W, b) = - \sum_{k} \mathbf{y}_k \space log(\mathbf{\hat{y}_k})
$$

* Gradient

$$
\frac{\partial J(W, b)}{\partial W} = X^T(\hat{Y} - Y) \\[10pt]
\frac{\partial J(W, b)}{\partial b} = \hat{Y} - Y
$$

In [ ]:
def fit(X, y, lr, n_classes, epochs):
    '''
    Args:
        X: Feature matrix
        y: Labels
        lr: Learning rate
        n_classes: Number of classes
        epochs: Number of epochs for training
    
    Returns:
        w: learned weight vectors
        b: learned biases
        Losses: Losses reported in each epoch

    '''
    n_samples, n_features = X.shape

    w = np.random.random((n_features, n_classes))
    b = np.random.random(n_classes)

    Losses = []

    for epoch in range(epochs):
        
        z = linear_combination(X, w, b)
        y_hat = softmax(z)
        y_hot = convert_to_one_hot_encoding(y, n_classes)

        w_grad = (1/n_samples) * np.dot(X.T, (y_hat - y_hot))
        b_grad = (1/n_samples) * np.sum(y_hat - y_hot)

        w -= lr * w_grad
        b -= lr * b_grad

        loss = -np.mean(np.log(y_hat[np.arange(len(y)), y]))
        Losses.append(loss)

        if (epoch % 10 == 0):
            print('Epoch {epoch} ==> Loss = {loss}'.format(epoch=epoch, loss=loss))

    return w, b, Losses

In [ ]:
def predict(X, w, b):
    '''
    Args:
        X: Feature matrix
        w: Weight vector
        b: Bias

    Return:
        Predictions
    '''

    z = linear_combination(X, w, b)
    y_hat = softmax(z)

    return np.argmax(y_hat, axis=1)

def accuracy(y, y_hat):
    return np.sum(y == y_hat)/len(y)

## Digit recognition on MNIST data

In [ ]:
weight, bias, losses = fit(X_train, train_y, lr=0.9, n_classes=10, epochs=100)

### Learning Curve

In [ ]:
plt.figure()
plt.title("Learning Curve")
plt.legend(['Training Loss'])
plt.xlabel('No. of Epochs')
plt.ylabel('Loss function')
plt.plot(np.arange(len(losses)), losses, 'b-')
plt.show()

In [ ]:
train_preds = predict(X_train, weight, bias)
print('Accuracy of training set: ', accuracy(train_y, train_preds))

In [ ]:
test_preds = predict(X_test, weight, bias)
print('Accuracy of test set: ', accuracy(test_y, test_preds))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

cm_disp = ConfusionMatrixDisplay.from_predictions(test_y, test_preds)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, test_preds))